In [1]:
# Loading dependencies
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm
import time

In [2]:
options = Options()
options.add_argument('--headless')
ua = {"User-Agent":"Mozilla/5.0"}
url = "https://www.whiskybase.com/whiskies/new-releases?bottle_date_year=2022&bottler_id=&brand_id=&vintage_year=&type_id=&itemsforsale=&votes="
url2 = "https://www.whiskybase.com/whiskies/top1000"
page = requests.get(url, headers=ua)
page2 = requests.get(url2, headers=ua)
soup = BeautifulSoup(page.text, "html")
soup2 = BeautifulSoup(page2.text, "html")
atags = soup.find_all('a', {'class' : 'clickable'})
atags2 = soup2.find_all('a', {'class' : 'clickable'})
links = [atag['href'] for atag in atags]
links2 = [atag2['href'] for atag2 in atags2]

In [3]:
list_of_dicts = []

def get_ratings_new_releases(links):
    url = "https://www.whiskybase.com/account/login?p=%2Fwhiskies%2Fnew-releases"
    driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
    driver.get(url)
    element = driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/div/div[3]/form/div[1]/div/input").send_keys("whiskie_coder")
    element = driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div/div[3]/form/div[2]/div/input").send_keys("whiskie_coder")
    element = driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div/div[3]/form/div[3]/div/button").send_keys(Keys.ENTER)
    
    for i in tqdm(links, desc="New Releases Progress"):
        url = i
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        try:
            price = soup.find('div', class_="block-price").find_all('p')[-1].get_text()
        except:
            price = 'N/A'
        
        community_blocks = soup.find_all('div', class_="wb--community-block")
        
        if len(community_blocks) > 1 and price != 'N/A':
            users = community_blocks[1].find_all('th', scope='row')
            user_list = []
            for user in users:
                try:
                    user_list.append(user.get_text().strip('\n'))
                except:
                    pass
            ratings = community_blocks[1].find_all('div', class_="wb--rating")
            ratings_list = []
            for rating in ratings:
                try:
                    ratings_list.append(rating.get_text().strip('\n').split('\n')[0])
                except:
                    pass
            for i in range(len(ratings_list)):
                try:
                    whiskey_dict = {}
                    whiskey_dict['whiskey'] = url.split('/')[-1]
                    whiskey_dict['user'] = user_list[i]
                    whiskey_dict['rating'] = ratings_list[i]
                    whiskey_dict['price'] = price
                    whiskey_dict['url'] = url
                    list_of_dicts.append(whiskey_dict)
                except:
                    pass
        else:
            # Pass the whiskeys with no ratings and price
            pass
    
    return list_of_dicts

In [4]:
list_of_dicts2 = []
def get_ratings_top1000(links2):
    url = "https://www.whiskybase.com/account/login?p=%2Fwhiskies%2Ftop1000"
    driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
    driver.get(url)
    element = driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/div/div[3]/form/div[1]/div/input").send_keys("whiskie_coder")
    element = driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div/div[3]/form/div[2]/div/input").send_keys("whiskie_coder")
    element = driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div/div[3]/form/div[3]/div/button").send_keys(Keys.ENTER)
    for i in tqdm(links2, desc="Top 1000 Progress"):  # This line was also changed from 'links' to 'links2'
        url = i
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        try:
            price = soup.find('div', class_="block-price").find_all('p')[-1].get_text()
        except:
            price = 'N/A'
        users = soup.find_all('div', class_="wb--community-block")[1].find_all('th' ,scope='row')
        user_list = []
        for user in users:
            try:
                user_list.append(user.get_text().strip('\n'))
            except:
                pass
        ratings = soup.find_all('div', class_="wb--community-block")[1].find_all('div' ,class_="wb--rating")
        ratings_list = []
        for rating in ratings:
            try:
                ratings_list.append(rating.get_text().strip('\n').split('\n')[0])
            except:
                pass
        for i in range(len(ratings_list)):
            try:
                whiskey_dict = {}
                whiskey_dict['whiskey'] = url.split('/')[-1]
                whiskey_dict['user'] = user_list[i]
                whiskey_dict['rating'] = ratings_list[i]
                whiskey_dict['price'] = price
                whiskey_dict['url'] = url
                list_of_dicts2.append(whiskey_dict)
            except:
                pass
    
    return list_of_dicts2

In [4]:
list_of_dicts = get_ratings_new_releases(links)
df = pd.DataFrame(list_of_dicts)
df.to_csv('Whiskey_Released_in_2022_Data.csv')

C:\Users\exman\AppData\Local\Temp\ipykernel_16788\233208415.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
New Releases Progress: 100%|██████████| 8221/8221 [1:12:23<00:00,  1.89it/s]


In [5]:
list_of_dicts2 = get_ratings_top1000(links2)
df2 = pd.DataFrame(list_of_dicts2)
df2.to_csv('Whiskey_Top_1000_Data.csv')

C:\Users\exman\AppData\Local\Temp\ipykernel_14104\3486401894.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
Top 1000 Progress: 100%|██████████| 1000/1000 [10:29<00:00,  1.59it/s]
